In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')
f_test = pd.read_csv('formula_test.csv.zip')
f_train = pd.read_csv('formula_train.csv.zip')

test_size = 0.2

In [ ]:
del f_train['critical_temp']

# соединяем признаки в один фрэйм
df_train = pd.concat([train, f_train], axis=1)
df_test = pd.concat([test, f_test], axis=1)

In [ ]:
df_train.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,material
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.2La1.8Cu1O4
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.1La1.9Ag0.1Cu0.9O4
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.1La1.9Cu1O4
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.15La1.85Cu1O4
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.3La1.7Cu1O4


In [ ]:
x = df_train.drop(columns=['critical_temp', 'material'])
y = df_train['critical_temp']

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(np.array(x), np.array(y).reshape(-1, 1), test_size=test_size)

x_train = torch.FloatTensor(x_train)
x_val = torch.FloatTensor(x_val)
y_train = torch.FloatTensor(y_train)
y_val = torch.FloatTensor(y_val)

In [ ]:
y_train.shape

torch.Size([13608, 1])

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
class MyNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(MyNet, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[-1], n_hidden_neurons)
        self.act1 = torch.nn.Sigmoid()

        # self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        # self.act2 = torch.nn.Sigmoid()

        # self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        # self.act3 = torch.nn.Sigmoid()

        self.fc4 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.act4 = torch.nn.Sigmoid()

        self.fc5 = torch.nn.Linear(n_hidden_neurons, 1)



    def forward(self, x):
        x = self.fc1(x)
        x = self.act1(x)

        # x = self.fc2(x)
        # x = self.act2(x)

        # x = self.fc3(x)
        # x = self.act3(x)

        x = self.fc4(x)
        x = self.act4(x)

        x = self.fc5(x)

        return x

my_net = MyNet(200)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mnist_net = my_net.to(device)
#list(mnist_net.parameters())

In [ ]:
optimizer = torch.optim.Adam(my_net.parameters(), lr=0.001)
x_train = x_train.to(device)
x_val = x_val.to(device)
y_train = y_train.to(device)
y_val = y_val.to(device)
for epoch_index in range(2000):
    optimizer.zero_grad()

    y_pred = my_net.forward(x_train)
    loss_val = loss(y_pred, y_train)

    loss_val.backward()

    optimizer.step()

    test_preds = my_net.forward(x_val)
    accuracy = (test_preds.argmax(dim=1) == y_val).float().mean()
    # print(accuracy)
    # print(my_net.forward(x_val).argmax(dim=1))

y_pred = my_net.forward(x_val)

In [ ]:
y_pred = my_net.forward(x_val)

In [ ]:
y_pred

tensor([[73.8663],
        [66.1037],
        [20.1812],
        ...,
        [ 4.3436],
        [81.6607],
        [13.2249]], grad_fn=<AddmmBackward0>)

In [ ]:
print(f'r2 = {r2_score(y_val.numpy(), y_pred.detach().numpy())}')
print(f'MSE = {(t := mean_squared_error(y_val.numpy(), y_pred.detach()))}')
print(f'RRMSE = {np.sqrt(t / np.sum(pow(y_pred.detach().numpy(), 2))) * 100}%')

pd.DataFrame(zip(y_pred.detach().numpy(), y_val.numpy()));

r2 = 0.7501462206416778
MSE = 292.3267822265625
RRMSE = 0.6626556627452374%


In [ ]:
x_test = torch.FloatTensor(np.array(df_test.drop(columns=['material'])))

temperature = my_net.forward(x_test)

In [ ]:
df_test['critical_temp'] = temperature.detach().numpy()

In [ ]:
df_test

In [ ]:
df_test['critical_temp'].to_csv('predict.csv', sep=';', index=False)